In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/train'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import Sequential
import keras, shutil
from tensorflow.keras.layers import Conv2D, ZeroPadding2D,Flatten,Dense, MaxPooling2D
from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import tensorflow

print("Tensorflow Version: {}".format(tensorflow.__version__))

Tensorflow Version: 2.6.2


In [4]:


# print num of samples in training and validation directory
train_num_samples = 0
for _,_,filenames in os.walk('/kaggle/input/textorientationdataset/text-orientation-data-train/train/'):
    train_num_samples += len(filenames) 
    
print('train_num_samples: ', train_num_samples) 

val_num_samples = 0
for _, _, filenames in os.walk('/kaggle/input/textorientationdataset/text-orientation-data-train/val/'):
    val_num_samples += len(filenames) 
    
print('val_num_samples: ', val_num_samples) 



train_num_samples:  4304
val_num_samples:  1400


In [5]:
# create Image Data Generator
datagen = ImageDataGenerator(rescale = 1.0/255) 

train_batch_size = 64
train_dir = '/kaggle/input/textorientationdataset/text-orientation-data-train/train/'
# create train generator
train_gen = datagen.flow_from_directory(train_dir, 
                                        target_size = (224, 224), 
                                        batch_size = train_batch_size, 
                                        class_mode = 'categorical') 

val_batch_size = 64
val_dir = '/kaggle/input/textorientationdataset/text-orientation-data-train/val/'
val_gen = datagen.flow_from_directory(val_dir, 
                                      target_size = (224, 224), 
                                      batch_size = val_batch_size, 
                                      class_mode = 'categorical') 

Found 4304 images belonging to 4 classes.
Found 1400 images belonging to 4 classes.


In [6]:
print(train_gen.class_indices)

{'0': 0, '180': 1, '270': 2, '90': 3}


In [ ]:
x, y = train_gen.next()
plt.figure(figsize = (13,13))

for i,(img, label) in enumerate(zip(x, y)):
    plt.subplot(4, 4,i+1) 
    if label == 0:
        plt.title('Rotated-0')
    elif label == 1:
        plt.title('Rotated-180')
    elif label == 2:
        plt.title('Rotated-270')
    else:
        plt.title('Rotated-90') 
        
    plt.axis('off') 
    plt.imshow(img) 

In [7]:
n_h = 224 # height of the image
n_w = 224 # width of the image
n_c = 3 # num of channels in image

# num of filters for convolutional layers
n_filters = [64,64,128,128,256,256,256,512,512,512,512,512,512]
len(n_filters) 

13

In [9]:
model = Sequential()

# create 2 conv layers with 64 filters of size 3,padding same and stride of 1
model.add(Conv2D(filters = n_filters[0], input_shape = (n_w, n_h, n_c), kernel_size = 3,
                 padding = 'SAME', activation='relu')) 
model.add(Conv2D(filters = n_filters[1],kernel_size = 3,
                 padding = 'SAME', activation='relu'))

# create max pooling layer
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

# create 2 conv layers with 128 filters
model.add(Conv2D(filters = n_filters[2],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[3],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
# create max pooling layer
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2))) 

# create 3 conv layers with 256 filters
model.add(Conv2D(filters = n_filters[4],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[5],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[6],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
# create max pooling layer
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2))) 

# create 3 conv layers with 512 filters

model.add(Conv2D(filters = n_filters[7],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[8],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[9],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
# create max pooling layer
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

# create 3 conv layers with 512 filters

model.add(Conv2D(filters = n_filters[10],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[11],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
model.add(Conv2D(filters = n_filters[12],kernel_size = 3,
                 padding = 'SAME', activation='relu'))
# create max pooling layer
model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

# flatten the layer
model.add(Flatten())

# create 2 fully connected layers with 4096 neurons
model.add(Dense(units = 4096, activation='relu')) 
model.add(Dense(units = 4096, activation='relu')) 

# final add the output layer
model.add(Dense(units = 4, activation = 'softmax')) 


# print model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 256)      

In [10]:
# compile the model
model.compile(loss='categorical_crossentropy', 
              optimizer = Adam(learning_rate = 0.00003), 
              metrics = ['accuracy']) 

# calculate train_steps and val_steps
train_steps = np.ceil(train_num_samples / train_batch_size) 
val_steps = np.ceil(val_num_samples / val_batch_size) 

# train the model
history = model.fit_generator(train_gen, 
                              steps_per_epoch = train_steps, 
                              epochs = 20,
                              validation_data = val_gen, 
                              validation_steps = val_steps) 

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-01-04 05:49:50.421513: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-01-04 05:49:53.862500: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


68/68 [==============================] - 130s 2s/step - loss: 1.3587 - accuracy: 0.3097 - val_loss: 1.1380 - val_accuracy: 0.4943
Epoch 2/20
68/68 [==============================] - 76s 1s/step - loss: 1.1581 - accuracy: 0.5072 - val_loss: 0.9467 - val_accuracy: 0.6136
Epoch 3/20
68/68 [==============================] - 77s 1s/step - loss: 0.9664 - accuracy: 0.6085 - val_loss: 0.7743 - val_accuracy: 0.6943
Epoch 4/20
68/68 [==============================] - 81s 1s/step - loss: 0.7418 - accuracy: 0.7168 - val_loss: 0.5939 - val_accuracy: 0.7879
Epoch 5/20
68/68 [==============================] - 87s 1s/step - loss: 0.6098 - accuracy: 0.7718 - val_loss: 0.5577 - val_accuracy: 0.7943
Epoch 6/20
68/68 [==============================] - 83s 1s/step - loss: 0.4553 - accuracy: 0.8325 - val_loss: 0.4134 - val_accuracy: 0.8486
Epoch 7/20
68/68 [==============================] - 81s 1s/step - loss: 0.3314 - accuracy: 0.8838 - val_loss: 0.3566 - val_accuracy: 0.8686
Epoch 8/20
68/68 [============

In [15]:
model.save('/kaggle/working/recognizing_cats_and_dogs_using_vgg16.h5')

In [13]:
# load the model
model = keras.models.load_model("/kaggle/working/recognizing_cats_and_dogs_using_vgg16.h5")

test_path = "/kaggle/input/textorientationdataset/text-orientation-data-train/val/"


# create test generator
test_gen = datagen.flow_from_directory(test_path,
                                      target_size = (224, 224),
                                      batch_size = 64,
                                      class_mode = None,
                                      shuffle = False)

# make predictions
preds = model.predict_generator(test_gen, 
                               steps = len(test_gen))

Found 1400 images belonging to 4 classes.


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [14]:
preds = np.squeeze(preds)

# create the dataframe
submission = pd.DataFrame({"files": test_files, "label": preds})
submission.head()

NameError: name 'test_files' is not defined